In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [25]:
url = 'https://sofifa.com/players?type=all&oah=70&ptl=80&vlh=1000000'
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")
html.status_code

200

In [26]:
productos = soup.find_all("td", {"class": "col-name"})

In [27]:
jugadores = {'jugador': []}

In [28]:
for i in productos:
    jugadores['jugador'].append(i.find('a').get('href'))   

In [29]:
jugadores_top = pd.DataFrame(jugadores)
jugadores_top.head()

,jugador
0,/player/267750/renzo-sanchez/220057/
1,/team/111325/club-nacional-de-football/
2,/player/266437/dimitrios-keramitsis/220057/
3,/team/52/roma/
4,/player/266031/ruben-iranzo-lendinez/220057/


In [30]:
jugadores_top['jug'] = jugadores_top['jugador'].str.extract(r'(?<=\/player\/)(.*)(?=\/220057)')

In [31]:
jugadores_top.head()

,jugador,jug
0,/player/267750/renzo-sanchez/220057/,267750/renzo-sanchez
1,/team/111325/club-nacional-de-football/,NaN
2,/player/266437/dimitrios-keramitsis/220057/,266437/dimitrios-keramitsis
3,/team/52/roma/,NaN
4,/player/266031/ruben-iranzo-lendinez/220057/,266031/ruben-iranzo-lendinez


In [32]:
jugadores_fichar = jugadores_top.dropna()
jugadores_fichar = jugadores_fichar.reset_index(drop=True)
jugadores_fichar.head()

,jugador,jug
0,/player/267750/renzo-sanchez/220057/,267750/renzo-sanchez
1,/player/266437/dimitrios-keramitsis/220057/,266437/dimitrios-keramitsis
2,/player/266031/ruben-iranzo-lendinez/220057/,266031/ruben-iranzo-lendinez
3,/player/264179/pelayo-gonzalez-rey/220057/,264179/pelayo-gonzalez-rey
4,/player/263640/antoni-djibu-milambo/220057/,263640/antoni-djibu-milambo


In [33]:
equipos = jugadores_fichar.drop(['jugador'], axis=1)
equipos.head()

,jug
0,267750/renzo-sanchez
1,266437/dimitrios-keramitsis
2,266031/ruben-iranzo-lendinez
3,264179/pelayo-gonzalez-rey
4,263640/antoni-djibu-milambo


In [34]:
jugar_caract = pd.DataFrame()
for i in range(0,55):
    jug = equipos['jug'][i]
    
    
    url2 = 'https://sofifa.com/player/' + jug + '/220057/'
    html2 = requests.get(url2)
    soup2 = BeautifulSoup(html2.content, "html.parser")
    html2.status_code
    
    productos2 = soup2.find_all("ul", {"class": "pl"})
    
    
    caract = {'jugador': []}
    for i in productos2:
        caract['jugador'].append(i) 
    
    caract = pd.DataFrame(caract)

    caract.rename(columns = {'jugador': jug}, inplace = True)
    caract[jug] = caract[jug].astype(str)
    
    caract[jug] = caract[jug].str.extract(r'(?<=\d\"\>)(\d+)(?=\<\/span\>)')
    
    caract_jug = caract.dropna()
    caract_jug = caract_jug.T

    
    jugar_caract = pd.concat([jugar_caract, caract_jug], axis=0)

In [35]:
jugar_caract = jugar_caract.reset_index(drop=True)

In [36]:
jugar_caract.head()

,2,3,4,5,6,7,8,9,10
0,70,64,63,62,77,60,30,35,13
1,79,64,37,32,55,42,59,59,7
2,77,64,44,38,59,53,69,57,8
3,69,65,42,60,66,60,69,54,9
4,75,64,56,69,70,60,44,41,15


In [37]:
nuevas_columnas = {
    2 : 'Valoración equipo actual', 
    3: "Mejor valoración individual", 
    4: "Ataque", 
    5: "Regate", 
    6: "Aceleración",
    7: 'Potencia tiro',
    8: 'Agresividad',
    9: 'Defensa',
    10: 'Portería'}

In [38]:
jugar_caract.rename(columns = nuevas_columnas, inplace = True)
jugar_caract.head()

,Valoración equipo actual,Mejor valoración individual,Ataque,Regate,Aceleración,Potencia tiro,Agresividad,Defensa,Portería
0,70,64,63,62,77,60,30,35,13
1,79,64,37,32,55,42,59,59,7
2,77,64,44,38,59,53,69,57,8
3,69,65,42,60,66,60,69,54,9
4,75,64,56,69,70,60,44,41,15


In [39]:
jugar_caract.to_csv('../../data/promesas/valoraciones_jugadores.csv')